# 네이버웹툰 크롤링
### chromedrive.exe의 위치는 이 주피터 노트북이 있는 디렉토리 안에 있어야 함
### [참고](https://yeeybook.tistory.com/139)

In [ ]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
import pandas as pd
import os
import sys

In [ ]:
URL = 'https://comic.naver.com/webtoon/weekday.nhn'
html = requests.get(URL).text # html 문서 전체를 긁어서 출력해줌, .text는 태그 제외하고 text만 출력되게 함
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
title = soup.find_all('a', {'class' : 'title'}) # a태그에서 class='title'인 html소스를 찾아 할당
id_list = []
title_list = []
author_list = []
day_list = []
genre_list = []
num = 0

In [ ]:
driver = webdriver.Chrome('chromedriver.exe')
driver.get(URL)

In [ ]:
for i in range(len(title)):
    sleep(1)
    page = driver.find_elements_by_class_name('title')
    page[i].click() #월요일 첫 번째 웹툰부터 순서대로 클릭
    sleep(1)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser') # 이동한 페이지 주소 읽고 파싱

    day = soup.find_all('ul', {'class' : 'category_tab'})
    day = day[0].find('li', {'class' : 'on'}).text[0:1] # 요일 수집

    t = title[i].text
    if (t in title_list):  # 요일 두 개 이상이면 요일만 추가함
        day_list[title_list.index(t)] += ', ' + day
        driver.back()
        continue
    id_list.append(i) ; num += 1  # id 리스트에 추가
    title_list.append(t)  # 제목 리스트에 추가
    day_list.append(day) # 요일 리스트에 추가

    author = soup.find_all('h2') # 두 번째 h2태그에 있음
    author = author[1].find('span', {'class' : 'wrt_nm'}).text[8:] # 7칸의 공백 후 8번 째부터 작가 이름임
    author_list.append(author) # 작가 리스트에 추가

    genre = soup.find('span', {'class' : 'genre'}).text # 장르 수집
    genre_list.append(genre) # 장르 리스트에 추가
    driver.back() # 뒤로 가기

In [ ]:
driver.close()

In [ ]:
cols = []
df = pd.DataFrame(columns = cols)
df['id'] = id_list
df['title'] = title_list
df['author'] = author_list
df['day'] = day_list
df['genre'] = genre_list
df.to_csv('네이버_웹툰.csv', encoding = 'utf-8-sig')